In [ ]:
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import seaborn as sns

from netCDF4 import Dataset
from tqdm import tqdm

%matplotlib inline


import os
path = r"<folder location>"      #replace <folder location>
direc_list = os.listdir(path)

date_from_name = [i.split('_')[-1].replace('1200.nc','') for i in direc_list]


#create latitude and longitude dataframe
def create_df_l(ds_name, ds_var, col_name):
    data = np.ma.getdata(ds_name[ds_var][:])
    df = pd.DataFrame(data)
    df = df.stack().reset_index().iloc[:,2:]
    df.rename(columns = {0:col_name}, inplace=True)
    return df

#create ice concentration dataframe
def create_df_c(ds_name, ds_var, col_name):
    data = np.ma.getdata(ds_name[ds_var][0,:,:])
    df = pd.DataFrame(data)
    df = df.stack().reset_index().iloc[:,2:]
    df.rename(columns = {0:col_name}, inplace=True)
    return df

#import data
keep_data = [Dataset(path + '\\' + i) for i in tqdm(direc_list)]

#create DataFrame from each file and keep them in a list
keep_df = []
for d,t in zip(tqdm(keep_data),date_from_name):
    df_lat = create_df_l(d, 'lat', 'lat')
    df_lon = create_df_l(d, 'lon', 'lon')
    df_ice = create_df_c(d,'ice_conc','ice_conc')
    
    df_con = pd.concat([df_lat,df_lon,df_ice], axis=1)
    
    keep_year = [int(t[0:4])]*len(df_con)
    keep_month = [int(t[4:6])]*len(df_con)
    df_con['year'] = keep_year
    df_con['month'] = keep_month
    
    df = df_con[(df_con['ice_conc']>=50) & (df_con['ice_conc']<=100)]
    keep_df.append(df)
    
#concat the list into a DataFrame
df = pd.concat(keep_df, axis=0)
df.reset_index(drop=True, inplace=True)


df_ac = df[(df['lat']>=66) & (df['ice_conc']>=0) & (df['ice_conc']<=100)]
df_des = df_ac.groupby(['year','month']).mean().reset_index()
df_des = df_des[['year','month','ice_conc']]

def mod_loca(df):
    df['lat_mod'] = [(((i-66)/24)*-1)+1 for i in df['lat']]
    df['lon_mod'] = [i+180 for i in df['lon']]
    return df

import itertools
list_month = list(range(1,13))
list_year = list(range(2010,2022))

keep_time = [list(i) for i in itertools.product(list_year,list_month)]
keep_time[0:9]


df_avg = df_ac.groupby(['year','month','lat', 'lon']).mean().reset_index()

df_ac_s = [df_ac[(df_ac['year']==i[0]) & (df_ac['month']==i[1])] for i in keep_time]
df_ac_m = [mod_loca(i) for i in df_ac_s]

keep_save_name = []     # list for saving output image file names 
for i,t in zip(df_ac_m, keep_time):
    sns.set_style('darkgrid')
    r = i['lat_mod']
    theta = i['lon_mod']

    fig, ax = plt.subplots(subplot_kw={'projection': 'polar'}, figsize = (9.5,9.5))
    sc = ax.scatter(theta, r, c = i['ice_conc'],
                    cmap='Blues',
                    alpha=0.65, vmin=50, vmax=100)
    ax.set_rmax(1.05)
    ax.set_rticks([])  # remove radial ticks
    ax.set_xticklabels(['', '', '', '', 'Prime\nMeridian', '', '', ''],fontdict={'size':15})
    ax.grid(True)
    ax.text(s = str(t[0]) + '_' + str(t[1]), x = 0.8, y = 1.19, fontdict={'size':15})

    ax.set_title("Average Percentage Concentration of Sea Ice above the Arctic Circle\n",
                 fontdict={'size':16})
    cbar = plt.colorbar(sc, fraction=0.03)
    cbar.set_label('% concentration of sea ice', fontdict={'size':14})
    
    #mark North Pole location
    ax.scatter(0, 0, color='White', s = 100, marker = '^')

    #North Pole text
    ax.text(0, 0, '  North Pole', horizontalalignment='left', size=14, color='White')
    
    #export as PNG files
    save_name = str(t[0]) + '_' + str(t[1]) + '.png'
    plt.savefig(save_name, bbox_inches='tight', pad_inches = 0)
    keep_save_name.append(save_name)


from PIL import Image
import imageio
img = []
for i in keep_save_name[-24:]:           # read PNG files
    myImage = Image.open(i)
    img.append(myImage)

#export the GIF file, output location can be changed
imageio.mimsave('ice.gif', img, duration=0.9)

In [ ]:
import folium
from folium import Map
from folium import plugins
from folium.plugins import HeatMap
from folium.plugins import HeatMapWithTime
import itertools

import io
from PIL import Image


#filter only above the Equator
df_acfull = df[df['lat']>=0]

#groupby the DataFrame and get the average of each month
df_avg = df_acfull.groupby(['year','month','lat', 'lon']).mean().reset_index()

#select year and month
df_ex = df_avg[(df_avg['year']==2021) & (df_avg['month']==12)


#create a list of list of latitude, longitude and ice concentration
data = [[lat,lon,ice] for lat,lon,ice in zip(df_ex.lat, df_ex.lon, df_ex.ice_conc)]

#create a list of color for plotting
n = 5
pal = list(sns.color_palette(palette='YlGnBu_r', n_colors=n).as_hex())
g_list = [(i+1)/n for i in list(range(n))]
dict_color = dict(zip(g_list, pal))


list_month = list(range(1,13))
list_year = list(range(2019,2022))

keep_time = [list(i) for i in itertools.product(list_year,list_month)]
keep_time[0:10]

#filter the average DataFrame and keep each month in a list
df_avg_list = [df_avg[(df_avg['year']==i[0]) & (df_avg['month']==i[1])] for i in keep_time]

#create a list of list of list of latitude, longitude and ice concentration
data =[[[lat,lon,ice] for lat,lon,ice in zip(df.lat,df.lon,df.ice_conc)] for df in df_avg_list]

#color dictionary
n = 5
pal = list(sns.color_palette(palette='Blues', n_colors=n).as_hex())
g_list = [(i+1)/n for i in list(range(n))]
dict_color = dict(zip(g_list, pal))


hm = folium.Map(location = [65,-30], zoom_start=3)

HeatMapWithTime(data, auto_play=True, gradient = dict_color,
                max_opacity=0.5, min_opacity=0.1).add_to(hm)

folium.LayerControl().add_to(hm)
hm.save('folium_s.html')
hm